# Überbuchungsproblem einer Ferienwohnanlage

## Aufgabe
Eine Ferienwohnanlage hat **80 Wohnungen**. Erfahrungsgemäß werden **15% der Buchungen** wieder storniert. Der Besitzer nimmt für die Pfingstferien **95 Buchungen** an. 

### Fragestellungen
1. Berechnen Sie die Wahrscheinlichkeit, dass zu viele Buchungen angenommen wurden.
2. Bestimmen Sie die Anzahl an Buchungen, die der Besitzer für die Pfingstferien annehmen sollte, damit die Wahrscheinlichkeit, dass zu viele Buchungen angenommen wurden, kleiner als **5%** ist.

Quelle: *Lambacher Schweizer. Mathematik Qualifikationsphase. Leistungskurs. S. 286. Niedersachsen. 2018*

*Der hier abgebildete Text, die Berechungen sowie das Python-Programm wurden vollständig von ChatGPT 4o erstellt.*

---

## Lösung zu (a): Wahrscheinlichkeit der Überbuchung

Da jede Buchung mit **15%** Wahrscheinlichkeit storniert wird, beträgt die Wahrscheinlichkeit, dass eine Buchung **nicht** storniert wird:

$$ p = 1 - 0.15 = 0.85 $$

Die Anzahl der tatsächlich ankommenden Gäste folgt einer **Binomialverteilung**:

$$ X \sim B(n=95, p=0.85) $$

Gesucht ist die Wahrscheinlichkeit, dass mehr als **80** Gäste erscheinen:

$$ P(X > 80) = 1 - P(X \leq 80) $$

Mit der kumulativen Verteilungsfunktion der Binomialverteilung ergibt sich:

$$ P(X > 80) = 1 - P(X \leq 80) \approx 1 - 0.458 = 0.542 $$

Die Wahrscheinlichkeit einer Überbuchung beträgt somit **ca. 54,2%**.

```python
from scipy.stats import binom

n_buchungen = 95  # Anzahl der angenommenen Buchungen
p_stornierung = 0.15  # Wahrscheinlichkeit einer Stornierung
n_wohnungen = 80  # Verfügbare Wohnungen

p_ueberbuchung = 1 - binom.cdf(n_wohnungen, n_buchungen, 1 - p_stornierung)
print(f"Wahrscheinlichkeit einer Überbuchung: {p_ueberbuchung:.4f}")
```

---

## Lösung zu (b): Maximale Anzahl an Buchungen bei max. 5% Überbuchungswahrscheinlichkeit

Nun soll die maximale Anzahl an Buchungen bestimmt werden, sodass die Wahrscheinlichkeit einer Überbuchung unter **5%** bleibt.

Gesucht ist also das größte **n**, für das gilt:

$$ P(X > 80) < 0.05 $$

wobei weiterhin gilt:

$$ X \sim B(n, p=0.85) $$

Wir testen verschiedene Werte für **n**, bis die Bedingung erfüllt ist:

- Für **n = 88** ergibt sich:

$$ P(X > 80) = 1 - P(X \leq 80) \approx 0.048 $$

- Für **n = 89** ergibt sich:

$$ P(X > 80) \approx 0.057 $$

Da für **n = 88** die Wahrscheinlichkeit gerade noch unter **5%** liegt, lautet die maximale Anzahl an Buchungen:

$$ n = 88 $$

```python
n_buchungen_optimal = 80  # Startwert

while True:
    p_ueberbuchung = 1 - binom.cdf(n_wohnungen, n_buchungen_optimal, 1 - p_stornierung)
    if p_ueberbuchung >= 0.05:
        break
    n_buchungen_optimal += 1

n_buchungen_optimal -= 1
print(f"Maximale Buchungen ohne Überbuchungsrisiko über 5%: {n_buchungen_optimal}")
```

---

## Verbindung zu Hypothesentests
Dieses Problem kann als **rechtsseitiger Hypothesentest** formuliert werden:

- Nullhypothese \( H_0 \): Die Anzahl der anreisenden Gäste überschreitet **nicht** die Kapazität von **80 Wohnungen**.
- Alternativhypothese \( H_1 \): Mehr als **80 Gäste** erscheinen.

Mit der Binomialverteilung kann die Wahrscheinlichkeit für die Annahme der Nullhypothese berechnet werden. Dies kann in Python mit einem Hypothesentest-Programm oder in GeoGebra visualisiert werden.

---

## Fazit
- Die Wahrscheinlichkeit einer Überbuchung bei **95 Buchungen** beträgt **ca. 54,2%**.
- Um das Überbuchungsrisiko unter **5%** zu halten, sollte der Besitzer **maximal 88 Buchungen** annehmen.
- Das Problem kann als rechtsseitiger Hypothesentest betrachtet werden.

---

📌 **Tipp für interaktive Experimente:**
- Verwenden Sie **GeoGebra** für die Visualisierung des Überbuchungsproblems.
- Nutzen Sie ein **Python-Programm mit Schiebereglern**, um die Buchungsanzahl und Stornierungsrate dynamisch anzupassen.
- Experimentieren Sie mit dem **Hypothesentest-Tool**, um die Wahrscheinlichkeiten direkt zu berechnen!

In [27]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import binom
import ipywidgets as widgets
from IPython.display import display, clear_output

plt.rcParams.update({
    'font.size': 16,          # Allgemeine Schriftgröße
    'axes.titlesize': 16,     # Größe der Titel (z.B. mit plt.title)
    'axes.labelsize': 16,     # Größe der Achsenbeschriftungen (xlabel, ylabel)
    'xtick.labelsize': 14,    # Größe der x-Achsenticks
    'ytick.labelsize': 14,    # Größe der y-Achsenticks
    'legend.fontsize': 12,    # Größe der Legendentexte
    'figure.titlesize': 12    # Größe des allgemeinen Figure-Titels (falls verwendet)
})

def binomial_test(n, p0, alpha, dxTicks, test_type="two-sided"):
    """
    Führt einen Binomialtest durch und zeigt die kritischen Werte und zugehörige grafische Darstellung.
    
    Parameter:
    - n (int): Stichprobengröße
    - p0 (float): Erfolgswahrscheinlichkeit unter H0
    - alpha (float): Signifikanzniveau (z.B. 0.05)
    - dxTicks (int): Abstand der Ticks auf der x-Achse
    - test_type (str): Art des Tests ("left", "right", "two-sided")
    """
    x = np.arange(0, n + 1)
    pmf_values = binom.pmf(x, n, p0)
    alpha_half = alpha / 2
    output = ""

    # Berechne Mittelwert und Standardabweichung
    mu = n * p0
    sigma = np.sqrt(n * p0 * (1 - p0))

    # Bestimme den Bereich für die x-Achse (5-Sigma-Umgebung)
    x_min = int(np.max([0, mu - 5 * sigma]))  # Mindestens 0
    x_max = int(np.min([n, mu + 5 * sigma]))  # Maximal n

    if test_type == "left":
        k_crit = next(k for k in range(n + 1) if binom.cdf(k, n, p0) > alpha) - 1
        output += f"Linksseitiger Test:\nKritischer Wert: k_crit = {k_crit}\nAblehnungsbereich: X <= {k_crit}\n"
        plot_binomial(x, pmf_values, rejection_left=k_crit, title="Linksseitiger Test", k_crit=k_crit, x_min=x_min, x_max=x_max, dxTicks=dxTicks)
    
    elif test_type == "right":
        k_crit = next(k for k in range(n + 1) if binom.cdf(k, n, p0) > 1 - alpha) + 1
        output += f"Rechtsseitiger Test:\nKritischer Wert: k_crit = {k_crit}\nAblehnungsbereich: X >= {k_crit}\n"
        plot_binomial(x, pmf_values, rejection_right=k_crit, title="Rechtsseitiger Test", k_crit=k_crit, x_min=x_min, x_max=x_max, dxTicks=dxTicks)
    
    elif test_type == "two-sided":
        k_crit_left = next(k for k in range(n + 1) if binom.cdf(k, n, p0) > alpha_half) - 1
        k_crit_right = next(k for k in range(n + 1) if binom.cdf(k, n, p0) > 1 - alpha_half) + 1
        output += f"Zweiseitiger Test:\nAblehnungsbereich: X <= {k_crit_left} oder X >= {k_crit_right}\n"
        plot_binomial(x, pmf_values, rejection_left=k_crit_left,
                      rejection_right=k_crit_right,
                      title="Zweiseitiger Test", k_crit_left=k_crit_left, k_crit_right=k_crit_right,
                      x_min=x_min, x_max=x_max, dxTicks=dxTicks)
    
    else:
        output += "Ungültiger Testtyp. Bitte 'left', 'right' oder 'two-sided' wählen."
    
    print(output)

def plot_binomial(x, pmf_values, rejection_left=None, rejection_right=None,
                  title="Binomialtest", k_crit=None, k_crit_left=None, k_crit_right=None,
                  x_min=None, x_max=None, dxTicks=1):
    """
    Erstellt eine grafische Darstellung der Binomialverteilung
    mit Ablehnungsbereichen und kritischen Wert-Labels.
    """
    plt.figure(figsize=(12, 6))
    plt.bar(x, pmf_values, color="skyblue", edgecolor='black', label="Binomialverteilung")
    
    if rejection_left is not None:
        plt.bar(x[:rejection_left + 1], pmf_values[:rejection_left + 1],
                color="red", edgecolor='black', label="Ablehnungsbereich (links)")
    if rejection_right is not None:
        plt.bar(x[rejection_right:], pmf_values[rejection_right:],
                color="red", edgecolor='black', label="Ablehnungsbereich (rechts)")
    
    plt.title(title)
    plt.xlabel("Anzahl Erfolge (k)")
    plt.ylabel("Wahrscheinlichkeit")
    
    # Beschränke die x-Achse auf den 5-Sigma-Bereich
    plt.xlim(x_min - 1, x_max + 1)
    
    # Setze die Ticks auf den Benutzer-Wert für den Abstand
    plt.xticks(np.arange(x_min, x_max + 1, dxTicks))
    
    plt.legend()
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.show()

def interactive_binomial_test():
    """Interaktive Oberfläche für den Binomialtest mit Widgets."""
    n_slider = widgets.IntSlider(value=20, min=1, max=1000, step=1,
                                 description=r'$n$:')
    p0_slider = widgets.FloatSlider(value=0.5, min=0.01, max=1.0, step=0.01,
                                    description=r'$p_0$:')
    alpha_slider = widgets.FloatSlider(value=0.05,min=0.01, max=0.5,
                                       step=0.005,description=r'$\alpha$:',
                                       readout_format='.3f')
    dxTicks_slider = widgets.IntSlider(value=1, min=1, max=100, step=1,
                                       description=r'$dx_{ticks}$:')
    test_type_dropdown = widgets.Dropdown(options=['left', 'right', 'two-sided'],
                                          value='two-sided', description='Testtyp:')
    output_area = widgets.Output()

    def update(*args):
        with output_area:
            clear_output(wait=True)
            binomial_test(n_slider.value, p0_slider.value, alpha_slider.value, dxTicks_slider.value, test_type_dropdown.value)
    
    # Initiale Berechnung anzeigen
    with output_area:
        binomial_test(n_slider.value, p0_slider.value, alpha_slider.value, dxTicks_slider.value, test_type_dropdown.value)
    
    n_slider.observe(update, 'value')
    p0_slider.observe(update, 'value')
    alpha_slider.observe(update, 'value')
    dxTicks_slider.observe(update, 'value')
    test_type_dropdown.observe(update, 'value')

    display(widgets.VBox([n_slider, p0_slider, alpha_slider, dxTicks_slider, test_type_dropdown, output_area]))

# Interaktive Benutzeroberfläche starten
interactive_binomial_test()
